## Definitions (undirected graphs)

* **Eulerian trail (open Eulerian path)**
  A walk that uses every edge **exactly once**, starting and ending at (possibly) different vertices.

  **Exists iff** the graph is connected when you ignore isolated vertices and it has **exactly two** odd-degree vertices (the trail starts/ends at those).

* **Eulerian circuit (Eulerian cycle)**
  A closed walk that uses every edge **exactly once** and starts/ends at the **same** vertex.

  **Exists iff** the graph is connected when you ignore isolated vertices and **all** vertices have **even** degree.

> Handshaking lemma reminder: the number of odd-degree vertices in any graph is even.

In [11]:
import networkx as nx

EDGE_FILE = "out.ucidata-zachary"  # keep this file next to the notebook

G = nx.read_edgelist(
    EDGE_FILE,
    comments="%",         # ignore header lines starting with '%'
    nodetype=int,
    create_using=nx.Graph()
)

print(f"|V|={G.number_of_nodes()}, |E|={G.number_of_edges()}")


|V|=34, |E|=78


In [12]:
# Basic Euler tests (NetworkX built-ins)
print("is_eulerian (circuit)?      ", nx.is_eulerian(G))        # all degrees even?
print("has_eulerian_path (trail)?  ", nx.has_eulerian_path(G))  # exactly 0 or 2 odd degrees?

# Inspect odd-degree vertices (for reporting)
odd = sorted([u for u, d in G.degree() if d % 2 == 1])
print(f"#odd-degree vertices = {len(odd)}")
print("odd-degree vertices  :", odd)

# (Optional) NetworkX also exposes a convenience:
print("is_semieulerian (trail only)?", nx.is_semieulerian(G))


is_eulerian (circuit)?       False
has_eulerian_path (trail)?   False
#odd-degree vertices = 12
odd-degree vertices  : [2, 5, 9, 11, 12, 14, 20, 24, 25, 26, 29, 34]
is_semieulerian (trail only)? False


In [13]:
# Transform to an Eulerian multigraph
H_circuit = nx.eulerize(G)
print("Augmented graph is Eulerian? ->", nx.is_eulerian(H_circuit))

# Report how many edges were added (counting multiplicity)
added_edges_count = H_circuit.number_of_edges() - G.number_of_edges()
print("Number of duplicated (added) edges:", added_edges_count)

# Which original edges were duplicated (u,v with multiplicity > 1)?
from collections import Counter

def duplicated_edges(multig):
    base = nx.Graph(multig)  # underlying simple graph
    dup = []
    for u, v in base.edges():
        k = multig.number_of_edges(u, v)
        if k > 1:
            dup.append(((u, v), k - 1))  # how many extra copies
    return sorted(dup)

print("Duplicated edges (edge, extra_copies):")
print(duplicated_edges(H_circuit))


Augmented graph is Eulerian? -> True
Number of duplicated (added) edges: 8
Duplicated edges (edge, extra_copies):
[((1, 9), 1), ((1, 12), 1), ((2, 20), 1), ((5, 11), 1), ((14, 34), 1), ((24, 34), 1), ((26, 25), 1), ((29, 34), 1)]


In [14]:
from itertools import combinations
from networkx.algorithms.matching import min_weight_matching

# 1) All-pairs shortest-path distances (NetworkX)
dist = dict(nx.all_pairs_shortest_path_length(G))

odd = [u for u, d in G.degree() if d % 2 == 1]

# 2) Pick trail endpoints among odd vertices.
#    Heuristic (good and simple): choose the farthest odd-odd pair.
s, t = max(
    ((u, v) for u, v in combinations(odd, 2)),
    key=lambda pair: dist[pair[0]][pair[1]]
)
print("Chosen trail endpoints:", (s, t))

# 3) Build complete graph on the remaining odd vertices with weights = shortest-path lengths
rest = [x for x in odd if x not in (s, t)]
K = nx.Graph()
K.add_nodes_from(rest)
for u, v in combinations(rest, 2):
    K.add_edge(u, v, weight=dist[u][v])

# 4) Minimum-weight perfect matching (NetworkX)
M = min_weight_matching(K, weight="weight")
print("Matched pairs (to duplicate paths):", sorted(tuple(sorted(p)) for p in M))

# 5) Create a MultiGraph with duplicated shortest paths for each matched pair
M_trail = nx.MultiGraph(G)
for u, v in M:
    path = nx.shortest_path(G, u, v)   # NetworkX shortest path
    for a, b in zip(path, path[1:]):
        M_trail.add_edge(a, b)         # duplicate once

print("Now semieulerian?", nx.is_semieulerian(M_trail))
print("Has Eulerian path?", nx.has_eulerian_path(M_trail))


Chosen trail endpoints: (5, 24)
Matched pairs (to duplicate paths): [(2, 20), (9, 11), (12, 14), (25, 26), (29, 34)]
Now semieulerian? True
Has Eulerian path? True


In [15]:
# Circuit on the eulerized graph
circuit_edges = list(nx.eulerian_circuit(H_circuit))
print("Eulerian circuit :", circuit_edges)

# Trail on the semi-eulerized graph
trail_edges = list(nx.eulerian_path(M_trail))
print("Eulerian trail :", trail_edges)


Eulerian circuit : [(1, 32), (32, 34), (34, 23), (23, 33), (33, 21), (21, 34), (34, 19), (19, 33), (33, 16), (16, 34), (34, 15), (15, 33), (33, 34), (34, 31), (31, 33), (33, 30), (30, 34), (34, 29), (29, 34), (34, 27), (27, 30), (30, 24), (24, 34), (34, 28), (28, 24), (24, 33), (33, 32), (32, 29), (29, 3), (3, 28), (28, 25), (25, 32), (32, 26), (26, 25), (25, 26), (26, 24), (24, 34), (34, 14), (14, 34), (34, 10), (10, 3), (3, 33), (33, 9), (9, 34), (34, 20), (20, 2), (2, 22), (22, 1), (1, 20), (20, 2), (2, 31), (31, 9), (9, 3), (3, 14), (14, 2), (2, 18), (18, 1), (1, 14), (14, 4), (4, 8), (8, 3), (3, 4), (4, 13), (13, 1), (1, 12), (12, 1), (1, 9), (9, 1), (1, 11), (11, 5), (5, 11), (11, 6), (6, 17), (17, 7), (7, 6), (6, 1), (1, 7), (7, 5), (5, 1), (1, 8), (8, 2), (2, 4), (4, 1), (1, 3), (3, 2), (2, 1)]
Eulerian trail : [(5, 1), (1, 2), (2, 3), (3, 1), (1, 4), (4, 2), (2, 8), (8, 1), (1, 6), (6, 7), (7, 1), (1, 9), (9, 1), (1, 11), (11, 5), (5, 7), (7, 17), (17, 6), (6, 11), (11, 1), (1